In [ ]:
# Importing the libraries

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
from io import BytesIO
import pycountry
import plotly.express as px
from scipy.stats import normaltest
from scipy.stats import f_oneway
import statsmodels.api as sm
import statsmodels.api as sm
import matplotlib.pyplot as plt
import praw
from textblob import TextBlob
from statsmodels.formula.api import ols
from scipy.stats import kruskal
from scipy.stats import wilcoxon
from dotenv import load_dotenv
from os import getenv
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

Create the Dataset

In [ ]:
url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/EI_ISBU_Q?format=TSV&compressed=true' # The url of the data
headers = {'Accept-Encoding': 'gzip'} # This is important to get the gzip file
response = requests.get(url, headers=headers) # Get the data from the url

buf = BytesIO(response.content) # Read the gzip file
f = gzip.GzipFile(fileobj=buf) # Unzip the gzip file
content = f.read() # Read the unzipped file

df = pd.read_csv(BytesIO(content), sep='\t') # Read the unzipped file as a dataframe
print(df.head())


In [ ]:
# Melt the dataframe so all the Quarters are in one column
df = pd.melt(df, id_vars=['freq,unit,s_adj,indic,nace_r2,geo\TIME_PERIOD'], var_name='Quarter', value_name='Value')

# The column freq,unit,s_adj,indic,nace_r2,geo\TIME_PERIOD is split into 6 columns
df[['freq','unit','s_adj','indic','nace_r2','geo\TIME_PERIOD']] = df['freq,unit,s_adj,indic,nace_r2,geo\\TIME_PERIOD'].str.split(',', expand=True)

# Drop the column freq,unit,s_adj,indic,nace_r2,geo\TIME_PERIOD and move Quarter and Value to the back
df = df.drop(['freq,unit,s_adj,indic,nace_r2,geo\\TIME_PERIOD'], axis=1)
df = df[['freq', 'unit', 's_adj', 'indic', 'nace_r2','geo\TIME_PERIOD','Quarter', 'Value']]

# Print the first 5 rows of the dataframe
df.head()

In [ ]:
df.info()

In [ ]:
# set values containing ":" to NaN
df.loc[df['Value'].str.contains(':'), 'Value'] = pd.np.nan

In [ ]:
# if the Value column has an alphabetical character remove it
df['Value'] = df['Value'].str.replace('[a-zA-Z]', '')

In [ ]:
#Convert the Value column to a float
df['Value'] = df['Value'].astype(float)

In [ ]:
df.head()

In [ ]:
countrycode_map = {c.alpha_2: c.alpha_3 for c in pycountry.countries} # Create a dictionary of country codes

#Uk is not a country code so we will change it to GB
df.loc[df['geo\\TIME_PERIOD'] == 'UK', 'geo\\TIME_PERIOD'] = 'GB'

df['Country_Codes'] = df['geo\\TIME_PERIOD'].map(countrycode_map) # Map the country codes to the dataframe


In [ ]:
# Print unique values of the Country_Codes column
print(df['Country_Codes'].unique())

In [ ]:
# Perform Normailty tests for the Value column grouped by geo\TIME_PERIOD, indic, and s_adj

df.set_index(['geo\\TIME_PERIOD', 'indic', 's_adj'], inplace=True)

# Perform normality test on the Value column grouped by geo\TIME_PERIOD, indic, and s_adj
grouped = df.groupby(['geo\\TIME_PERIOD', 'indic', 's_adj'])['Value']
pvalues = grouped.apply(lambda x: normaltest(x.dropna())[1])

# Add p-values to a new column in the original DataFrame
df.loc[pvalues.index, 'pvalue'] = pvalues

#Add a new column to the dfi dataframe called 'normal' and set it to True if the pvalue is greater than 0.05 and False if it is less than 0.05
df['normal'] = df['pvalue'] > 0.05

# Move the index back to columns
df.reset_index(inplace=True)

Explore the Data

In [ ]:
# Create a country plot with the Value column as the color and the Quarter column as the animation, show for indicators '[IS-IP]' and Seasonal Adjustment 'NSA'

fig = px.choropleth(df[(df['indic'] == 'IS-IP') & (df['s_adj'] == 'NSA')], locations="Country_Codes",color="Value", hover_name="geo\TIME_PERIOD", animation_frame="Quarter", color_continuous_scale=px.colors.sequential.Plasma, range_color=(0, 100), scope='europe') # Create the plot

fig.show()

In [ ]:
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
# Group the dataframe by geo\TIME_PERIOD, indic, and s_adj and show descriptive statistics
df.groupby(['s_adj', 'indic', 'geo\\TIME_PERIOD']).describe(include='all')

In [ ]:
# Seperate the geo\TIME_PERIOD value that equals 'IE' into a seperate dataframe
df_ie = df[df['geo\TIME_PERIOD'] == 'IE']

In [ ]:
#Perform EDA on the IE dataframe
df_ie.info()

In [ ]:
df_ie.describe(include='all')

In [ ]:
# Perform Statistical Analysis on the IE dataframe
df_ie.describe(include='all')

df_ie.groupby(['indic','s_adj']).describe()

In [ ]:
# Create histomgrams of the Value column for the IE dataframe grouped by 'indic' and 's_adj', faceted by the 'normal' column
sns.FacetGrid(df_ie, col='indic', row='s_adj', hue='normal').map(sns.histplot, 'Value').add_legend()

In [ ]:
# Create histograms of the Value column for the IE dataframe grouped by 'indic' and 's_adj', where the 'normal' column is True
sns.FacetGrid(df_ie[df_ie['normal'] == True], row='indic', col='s_adj', hue= 's_adj').map(sns.histplot, 'Value').add_legend()

In [ ]:
# Visualise the IE dataframe showing statistical analysis using seaborn  and matplotlib  libraries splitting the data by insdicators and seasonanal adjustment
sns.catplot(x="indic", y="Value", hue="s_adj", kind="box", data=df_ie);

Perform Statistical Tests

In [ ]:
# return the unique geo\TIME_PERIOD values from the df dataframe, where the indic column equals 'IS-EPI' and the normal column equals True
df[(df['indic'] == 'IS-EPI') & (df['normal'] == True)]['geo\\TIME_PERIOD'].unique()

In [ ]:
#Compare Ireland with 'EU28' which is the overall EU average for 28 countries

# Create a dataframe called df_ie_eu28 that contains the geo\TIME_PERIOD values of 'IE' and 'EU28'
df_ie_eu28 = df[(df['geo\\TIME_PERIOD'] == 'IE') | (df['geo\\TIME_PERIOD'] == 'EU28')]

Perform a T-test

In [ ]:
# Compare the Number of persons employed index, non-seasonally adjusted, for Ireland and EU28 using a t-test
from scipy.stats import ttest_ind

# Filter the DataFrame to only include rows with geo\TIME_PERIOD values of 'EU28' and 'IE',
# 'IS-EPI' for indic, and 'NSA' for s_adj
df_filtered = df[(df['geo\\TIME_PERIOD'].isin(['EU28', 'IE'])) & (df['indic'] == 'IS-EPI') & (df['s_adj'] == 'NSA')]

# Filter out NaN values from the 'Value' column
df_filtered = df_filtered.dropna(subset=['Value'])

# Split the DataFrame into two separate DataFrames, one for each geo\TIME_PERIOD value
df_eu281 = df_filtered[df_filtered['geo\\TIME_PERIOD'] == 'EU28']
df_ie1 = df_filtered[df_filtered['geo\\TIME_PERIOD'] == 'IE']

# Perform the t-test
t_stat, p_val = ttest_ind(df_eu281['Value'], df_ie1['Value'], equal_var=False)

print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_val:.4f}")


Perform a One-Way ANOVA

In [ ]:
# Filter the dataframe to only include rows where the normal column equals True and the indic column equals 'IS-EPI' and the s_adj column equals 'NSA'
df_filtered = df[(df['normal'] == True) & (df['indic'] == 'IS-EPI') & (df['s_adj'] == 'NSA')]

# Filter out NaN values from the 'Value' column
df_filtered = df_filtered.dropna(subset=['Value'])

# Create a list of the unique geo\TIME_PERIOD values
geo_time_periods = df_filtered['geo\\TIME_PERIOD'].unique()

#Make sure the groups have the same sample size
min_sample_size = df_filtered['geo\\TIME_PERIOD'].value_counts().min()
df_filtered = df_filtered.groupby('geo\\TIME_PERIOD').apply(lambda x: x.sample(min_sample_size))

#Perform the ANOVA
f_stat, p_val = f_oneway(*[df_filtered[df_filtered['geo\\TIME_PERIOD'] == geo_time_period]['Value'] for geo_time_period in geo_time_periods])

print(f"F-statistic: {f_stat:.4f}")
print(f"p-value: {p_val:.4f}")

#Print th resuts of the ANOVA test
if p_val < 0.05:
    print("Reject null hypothesis - Significant differences exist between groups.")
else:
    print("Accept null hypothesis - No significant difference between groups.")

In [ ]:
#Show the diferences between the groups using a boxplot with slanted x-axis labels
sns.catplot(x="geo\\TIME_PERIOD", y="Value", kind="box", data=df_filtered).set_xticklabels(rotation=30)

In [ ]:
# Perform a Tukey's Range Test to determine which groups are significantly different from each other
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Perform Tukey's Range Test
tukey_results = pairwise_tukeyhsd(df_filtered['Value'], df_filtered['geo\\TIME_PERIOD'], 0.05)

# Print the results
print(tukey_results)

Perform a Two-Way ANOVA

In [ ]:
# Create a dataframe called prep1 where the indic column equals 'IS-EPI' and the s_adj column contains 'SCA and 'NSA' values and the normal column equals True
prep1 = df[(df['indic'] == 'IS-EPI') & (df['normal'] == True)]

In [ ]:
#Change the name of the geo\TIME_PERIOD column to geo in the df dataframe
prep1.rename(columns={'geo\\TIME_PERIOD': 'geo'}, inplace=True)

In [ ]:
# Perform a TWO-WAY ANOVA to determine if there is an interaction between the geo\TIME_PERIOD and s_adj columns

# Perform two-way ANOVA
model = ols('Value ~ s_adj + geo', data = prep1).fit()
aov2 = sm.stats.anova_lm(model, type=2)
print(aov2)

Perform a Wilcoxon Signed-Rank Test

In [ ]:
# Find countries that have false values for the normal column where the indic column equals 'IS-HWI'
df[(df['normal'] == False) & (df['indic'] == 'IS-HWI')]['geo\\TIME_PERIOD'].unique()

In [ ]:
# Filter the data
df_HWI = df[(df['indic'] == 'IS-HWI')]

# Filter out NaN values from the 'Value' column
df_HWI = df_HWI.dropna(subset=['Value'])


In [ ]:

# Select the two groups to compare
group1 = df_HWI[df_HWI['geo\\TIME_PERIOD'] == 'IE']['Value']
group2 = df_HWI[df_HWI['geo\\TIME_PERIOD'] == 'EU28']['Value']

# Ensure the two groups have the same sample size
min_sample_size = min(len(group1), len(group2))
group1 = group1.sample(min_sample_size)
group2 = group2.sample(min_sample_size)

# Perform Wilcoxon signed-rank test
stat, p = wilcoxon(group1, group2)

# Print the results
print('Wilcoxon signed-rank test:')
print(f'statistic: {stat:.4f}')
print(f'p-value: {p:.4f}')


Perform a Kruskall Wallis Test

In [ ]:
# Filter the data
df_filtered = df[(df['indic'] == 'IS-HWI') & (df['normal'] == False) & (df['s_adj'] == 'NSA')]

# Filter out NaN values from the 'Value' column
df_filtered = df_filtered.dropna(subset=['Value'])

# Perform Kruskal-Wallis test
stat, p = kruskal(*[df_filtered[df_filtered['geo\\TIME_PERIOD'] == geo]['Value'] for geo in df_filtered['geo\\TIME_PERIOD'].unique()])

# Print the results
print("Kruskal-Wallis Test Results:")
print(f"Test statistic: {stat:.4f}")
print(f"P-value: {p:.4f}")


Machine Learning

Sentiment Analysis

In [ ]:
# Load from the .env
load_dotenv()

# Get the environmental variables
APP_NAME = getenv('APP_NAME')
APP_ID = getenv("APP_ID")
APP_SECRET = getenv("APP_SECRET")
USERNAME = getenv('REDDIT_USERNAME')
PASSWORD = getenv('PASSWORD')

In [ ]:
# Set up reddit API credentials
reddit = praw.Reddit(
    client_id=APP_ID,
    client_secret=APP_SECRET,
    user_agent=APP_NAME,
    username=USERNAME,
    password=PASSWORD,
)

# Define the subreddits and search query
subreddits = ["Ireland", "Europe"]
query = "house prices"

# Collect posts from the subreddits related to the search query
posts = []
for subreddit_name in subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    for post in subreddit.search(query):
        posts.append(
            {
                "subreddit": subreddit_name,
                "title": post.title,
                "text": post.selftext,
            }
        )

# Convert the collected posts into a dataframe
df_posts = pd.DataFrame(posts)

# Perform sentiment analysis on the collected posts
df_posts["polarity"] = df_posts["text"].apply(lambda x: TextBlob(x).sentiment.polarity)

# Group the results by subreddit and calculate the mean polarity
results = df_posts.groupby("subreddit")["polarity"].mean()

# Print the results
print(results)


In [ ]:
# Interpret the results of the sentiment analysis
if results["Ireland"] > results["Europe"]:
    print("The sentiment of the posts from r/Ireland is more positive than the sentiment of the posts from r/Europe.")
elif results["Ireland"] < results["Europe"]:
    print("The sentiment of the posts from r/Ireland is more negative than the sentiment of the posts from r/Europe.")
else:
    print("The sentiment of the posts from r/Ireland is the same as the sentiment of the posts from r/Europe.")


Machine Learning Model 1

In [ ]:
#Time Series Analysis

# Create a dataframe called df_filtered where the indic column equals 'IS-HWI' and the s_adj column contains 'NSA' values and the normal column equals False
df_filtered = df[(df['indic'] == 'IS-HWI') & (df['normal'] == False) & (df['s_adj'] == 'NSA') & (df['geo\\TIME_PERIOD'] == 'IE')]
df_filtered = df_filtered.dropna(subset=['Value'])

# Define a function to parse the date string
def parse_quarter(date_string):
    year, quarter = date_string.split('-Q')
    month = (int(quarter) - 1) * 3 + 1
    return datetime(int(year), month, 1)

#Apply the parse_quarter function to the 'Quarter' column
df_filtered['Quarter'] = df_filtered['Quarter'].apply(parse_quarter)

In [ ]:
# Set the 'Quarter' column as the index
df_filtered.set_index('Quarter', inplace=True)

# Create a time series plot of the 'Value' column
df_filtered['Value'].plot(figsize=(12, 5))

# Add a title
plt.title('Hours Worked Index in Ireland')

# Add a y-axis label
plt.ylabel('Hours Worked Index')

# Show the plot
plt.show()

In [ ]:
df_filtered = df_filtered.asfreq('QS-OCT')

In [ ]:
from sklearn.metrics import mean_absolute_error
# Perform time series analysis using SARIMAX
# Split the data into train and test sets
train = df_filtered.iloc[:len(df_filtered) - 4] # everything up to the last 4 observations
test = df_filtered.iloc[len(df_filtered) - 4:]  # the last 4 observations

# Create a SARIMAX model
model = sm.tsa.statespace.SARIMAX(train['Value'])
fit_model = model.fit()

# Generate predictions
predictions = fit_model.predict(start=len(train), end=len(train) + len(test) - 1)

# Print the predictions
print(predictions)

# Plot the predictions
predictions.plot(figsize=(12, 5))

# Add a title
plt.title('Predictions for the last 4 quarters')

# Add a y-axis label
plt.ylabel('Hours Worked Index')

# Show the plot
plt.show()

# Print the test data
print(test)

# Print the mean absolute error (MAE)
print('The MAE is', mean_absolute_error(test['Value'], predictions))

# Print the root mean squared error (RMSE)
print('The RMSE is', mean_squared_error(test['Value'], predictions, squared=False))

In [ ]:
import itertools
from itertools import product
# Tune the models hyperparameters to improve the RMSE
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 4) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

# Find the optimal set of parameters that yields the best performance
# Define the initial parameters
best_score, best_params, best_seasonal_params = float("inf"), None, None

# Loop through the parameter combinations
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            # Create a SARIMAX model
            model = sm.tsa.statespace.SARIMAX(train['Value'], order=param, seasonal_order=param_seasonal)

            # Fit the model
            results = model.fit()

            # Generate predictions
            predictions = results.predict(start=len(train), end=len(train) + len(test) - 1)

            # Calculate the mean squared error
            mse = mean_squared_error(test['Value'], predictions)

            # If the mse is lower than our best score, update the best score, and best parameters
            if mse < best_score:
                best_score, best_params, best_seasonal_params = mse, param, param_seasonal

            # Print the model parameters and the mean squared error
            print('SARIMA{}x{}4 - AIC:{}'.format(param, param_seasonal, mse))
        except:
            continue
        
# Print the best model parameters and the mean squared error
print('Best SARIMA{}x{}4 AIC:{}'.format(best_params, best_seasonal_params, best_score))

In [ ]:
# Create a SARIMAX model
model1 = sm.tsa.statespace.SARIMAX(train['Value'], order=(1, 0, 0), seasonal_order=(0, 0, 1, 4))
fit_model1 = model1.fit()

# Generate predictions
predictions1 = fit_model1.predict(start=len(train), end=len(train) + len(test) - 1)

# Print the predictions
print(predictions1)

# Plot the predictions
predictions1.plot(figsize=(12, 5))

# Add a title
plt.title('Predictions for the last 4 quarters')

# Add a y-axis label
plt.ylabel('Hours Worked Index')

# Show the plot
plt.show()

# Print the test data
print(test)

# Print the mean absolute error (MAE)
print('The MAE is', mean_absolute_error(test['Value'], predictions))

# Print the root mean squared error (RMSE)
print('The RMSE is', mean_squared_error(test['Value'], predictions, squared=False))

In [ ]:
# Tune the model
model = sm.tsa.statespace.SARIMAX(train['Value'],order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
results = model.fit()

# Generate predictions
predictions = fit_model.predict(start=len(train), end=len(train) + len(test) - 1)

# Print the predictions
print(predictions)

# Plot the predictions
predictions.plot(figsize=(12, 5))

# Add a title
plt.title('Predictions for the last 4 quarters')

# Add a y-axis label
plt.ylabel('Hours Worked Index')

# Show the plot
plt.show()

# Print the test data
print(test)

# Print the mean absolute error (MAE)
print('The MAE is', mean_absolute_error(test['Value'], predictions))

# Print the root mean squared error (RMSE)
print('The RMSE is', mean_squared_error(test['Value'], predictions, squared=False))

In [ ]:
# Visualize the results
results.plot_diagnostics(figsize=(12, 10))
plt.show()

In [ ]:
# Get the predicted values
pred = results.predict()

# Plot the actual values and the predicted values
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(df_filtered['Value'], label='Actual')
ax.plot(pred, label='Predicted')
ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('Value')
plt.show()

In [ ]:
# Create a prediction for the next 4 quarters
pred = results.predict(start=len(df_filtered)-4, end=len(df_filtered) + 3)

# Print the predictions
print(pred)

In [ ]:
# Plot the Current Year with actual and predicted values and the Next Year with predicted values
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(df_filtered['Value'], label='Actual')
ax.plot(pred, label='Predicted')
ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('Value')

In [ ]:
#Time Series Analysis across countries

In [ ]:
# Create a dataframe called df_filtered where the indic column equals 'IS-HWI' and the s_adj column contains 'NSA' values and the normal column equals False
df_filteredall = df[(df['indic'] == 'IS-HWI') & (df['normal'] == False) & (df['s_adj'] == 'NSA')]
df_filteredall = df_filteredall.dropna(subset=['Value'])

# Define a function to parse the date string
def parse_quarter(date_string):
    year, quarter = date_string.split('-Q')
    month = (int(quarter) - 1) * 3 + 1
    return datetime(int(year), month, 1)

#Apply the parse_quarter function to the 'Quarter' column
df_filteredall['Quarter'] = df_filteredall['Quarter'].apply(parse_quarter)

# Set the style of the plots
sns.set(style='darkgrid')

# Use the 'FacetGrid' function to create facetted plots
g = sns.FacetGrid(df_filteredall, col='Country_Codes', col_wrap=3, height=4)

# Map the line plot onto the facetted plots
g.map(sns.lineplot, 'Quarter', 'Value')

# Set the x-axis label for each plot
g.set_axis_labels('Quarter', 'Value')

# Set the title for each plot
g.set_titles('{col_name}')

# Adjust the spacing between the plots
g.tight_layout()

# Show the facetted plots
plt.show()


In [ ]:
#index the dataframe by the 'Quarter' column
df_filtered.set_index('Quarter', inplace=True)

In [ ]:
def fit_sarimax_model(data):
    # Split the data into train and test sets (e.g., 80% train, 20% test)
    train_data = data[:-4]  # Use all but the last 4 quarters for training
    test_data = data[-4:]  # Use the last 4 quarters for testing
    
    # Fit the SARIMAX model
    model = sm.tsa.statespace.SARIMAX(train_data, order=(1, 0, 0), seasonal_order=(1, 0, 0, 4))
    model_fit = model.fit()

    # Make predictions for the test set
    predictions = model_fit.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1)

    # Return the model fit and predictions
    return model_fit, predictions

In [ ]:
results = {}  # Dictionary to store the results

# Iterate over each country
for country in df_filteredall['geo\\TIME_PERIOD'].unique():
    # Filter the data for the current country and reset the index
    country_data = df_filteredall[df_filteredall['geo\\TIME_PERIOD'] == country]['Value'].reset_index(drop=True)

    # Apply differencing to make the series stationary
    country_data_diff = country_data.diff().dropna()

    # Fit SARIMAX model and make predictions
    model_fit, predictions = fit_sarimax_model(country_data_diff)

    # Store the model fit and predictions for the country
    results[country] = {'model_fit': model_fit, 'predictions': predictions}


Machine Learning Model 2

Support Vector Regression

In [ ]:
# Filter the data
df_drop = df.dropna(subset=['Value'])
df_drop = df_drop[(df_drop['indic'] == 'IS-EPI')]

# Scale the 'Value' column
df_drop['Value'] = StandardScaler().fit_transform(df_drop[['Value']])

# select the relevant columns as features and target
X = df_drop[['geo\\TIME_PERIOD', 's_adj']]
y = df_drop['Value']

# perform one-hot encoding on the categorical columns
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2)

# create and fit the model
model = SVR()
model.fit(X_train, y_train)

# make predictions on the test set
y_pred = model.predict(X_test)

# calculate the model's performance metrics
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE:", mse)
print("R-squared:", r2)



In [ ]:
# create a Support Vector Regression object
svr = SVR()

# define the hyperparameter grid to search over
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly']}

# create a GridSearchCV object
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# print the best hyperparameters found
print(grid_search.best_params_)


In [ ]:
# Use the best hyperparameters to create a new model
best_svr = SVR(C=100, gamma=0.1, kernel='linear')

# Fit the model to the training data
best_svr.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_svr.predict(X_test)

# Calculate the model's performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE:", mse)
print("R-squared:", r2)

In [ ]:
import matplotlib.pyplot as plt

# Plot the actual values and predicted values
plt.scatter(y_test, y_pred, color='blue')

# Add a diagonal line for comparison
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)

# Add labels and title
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('SVR Predictions vs. Actual Values')

# Show the plot
plt.show()

Create a Dashboard to show Results

In [ ]:
# Filter the DataFrame based on the conditions
df_filtered = df[(df['indic'] == 'IS-HWI') & (df['normal'] == False) & (df['s_adj'] == 'NSA')].copy()
df_filtered.dropna(subset=['Value'], inplace=True)

# Drop the rows with null values in Country_Codes column
df_filtered = df_filtered.dropna(subset=["Country_Codes"])

In [ ]:
# Add a new column to df_filtered called Value_Pred, with is the Value column + 10 - This will later be the predicted time series values.
df_filtered["Value_Pred"] = df_filtered["Value"] + 10

In [ ]:
# Create a dictionary to map country codes to full names
country_names = {
 "NLD": "Netherlands",
 "LTU": "Lithuania",
 "BEL": "Belgium",
 "BGR": "Bulgaria",
 "CYP": "Cyprus",
 "EST": "Estonia",
 "ESP": "Spain",
 "FIN": "Finland",
 "HRV": "Croatia",
 "IRL": "Ireland",
 "ITA": "Italy",
 "LVA": "Latvia",
 "MNE": "Montenegro",
 "MLT": "Malta",
 "NOR": "Norway",
 "PRT": "Portugal",
 "SWE": "Sweden",
 "TUR": "Turkey",
 "MKD": "North Macedonia"
}

# Create a new column in df_filtered with full country names
df_filtered["Country_Names"] = df_filtered["Country_Codes"].map(country_names)

In [ ]:
#Create dashboard layout
# Create initial figures
fig_line = px.line(df_filtered, x="Quarter", y="Value", color="Country_Codes") 
fig_map = px.choropleth(df_filtered, locations="Country_Codes", color="Value", scope="europe") 

# Add another line to fig_line based on Value_Pred column
fig_line.add_trace(go.Scatter(x=df_filtered["Quarter"], y=df_filtered["Value_Pred"], name="Predicted Value", mode="lines", line=dict(color="black", dash="dash")))

# Create app layout
app = JupyterDash(__name__) # Use JupyterDash instead of Dash
app.layout = html.Div([
 html.H1("Dashboard Example"),
 dcc.RadioItems( 
 id="country-buttons", 
 options=[{"label": c, "value": c} for c in df_filtered["Country_Names"].unique()], # Use full country names as labels
 value=df_filtered["Country_Names"].iloc[0],
 style={"table-layout": "auto"}, # Style the container element 
 labelStyle={"display": "inline-block", "margin-right": "10px"} # Style each label element
 ),
 html.Div([ # Wrap line chart and map in a div with flex display
     html.Div([ # Wrap dropdown in a div with 50% width
         dcc.Graph(id="map-chart", figure=fig_map)
     ], style={"width": "50%"}),
     html.Div([ # Wrap map in a div with 50% width
         dcc.Graph(id="line-chart", figure=fig_line)
     ], style={"width": "50%"})
 ], style={"display": "flex", "flex-direction": "row"}),
])

# Define callback function
@app.callback(
 [Output("line-chart", "figure"), Output("map-chart", "figure")],
 [Input("country-buttons", "value")]
)
def update_charts(country_name):
 # Filter dataframe by selected country name
 df_country = df_filtered[df_filtered["Country_Names"] == country_name] 
 # Create new figures
 fig_line = px.line(df_country, x="Quarter", y="Value", color="Country_Codes") 
 fig_map = px.choropleth(df_country, locations="Country_Codes", color="Value", scope="europe") 
 # Add another line to fig_line based on Value_Pred column
 fig_line.add_trace(go.Scatter(x=df_country["Quarter"], y=df_country["Value_Pred"], name="Predicted Value", mode="lines", line=dict(color="black", dash="dash")))
 # Return new figures
 return fig_line, fig_map

# Run app
app.run_server(mode="external") # Set mode to "inline" or "external"